#### NOTA: En este cuaderno se crearan las Tablas `localidad`, `provincia`, <br> `proveedor`, `cliente` y `sucursal` del Datawarehouse

***
### Importando Librerias

In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.types import IntegerType, FloatType

In [2]:
from pyspark.sql.functions import count, when, isnull, monotonically_increasing_id, initcap
from pyspark.sql.functions import initcap, when, col, regexp_replace, coalesce, avg

***
Creando Session de Spark

In [3]:
spark = SparkSession.builder.appName('localidad-provincia-etc').getOrCreate()

***
Importando Conexion a Contenedor `mod4_mysql`

In [4]:
from db.connection import connec_db

***
Trabajando Las Tablas `proveedor`, `sucursal` y `cliente` 

In [4]:
proveedor = spark.read.csv("Data/Proveedores.csv", sep=",", header=True, inferSchema=True)

In [5]:
sucursal = spark.read.csv("Data/Sucursales.csv", sep=";", header=True, inferSchema=True)

In [6]:
cliente = spark.read.csv("Data/Clientes.csv", sep=";", header=True, inferSchema=True)

***
Visualizando como vienen los Datos

In [7]:
proveedor.show(5)

+-----------+-------------------+--------------------+----------------+------------+---------+----------+
|IDProveedor|             Nombre|             Address|            City|       State|  Country|departamen|
+-----------+-------------------+--------------------+----------------+------------+---------+----------+
|          1|               NULL|     BEREGUESTEIN 36|      AVELLANEDA|BUENOS AIRES|ARGENTINA|AVELLANEDA|
|          2|         San Cirano|         ALBERTI 965|CORONEL BRANDSEN|BUENOS AIRES|ARGENTINA|  BRANDSEN|
|          3|          Bell S.A.|          MORENO 331|         CAMPANA|BUENOS AIRES|ARGENTINA|   CAMPANA|
|          4|     Rivero Insumos|     PTE. PERON 3872| VALENTIN ALSINA|BUENOS AIRES|ARGENTINA|     LANUS|
|          5|Laprida Computacion|AVDA. PEDRO PEREY...|         LAPRIDA|BUENOS AIRES|ARGENTINA|   LAPRIDA|
+-----------+-------------------+--------------------+----------------+------------+---------+----------+
only showing top 5 rows



In [8]:
sucursal.show(5)


+---+----------+--------------------+--------------------+--------------------+-----------+-----------+
| ID|  Sucursal|           Direccion|           Localidad|           Provincia|    Latitud|   Longitud|
+---+----------+--------------------+--------------------+--------------------+-----------+-----------+
|  1|   Cabildo|    Av. Cabildo 1342|Ciudad de Buenos ...|Ciudad de Buenos ...|-34,5678060|-58,4495720|
|  2| Palermo 1|      Guatemala 5701|                CABA|                CABA|-34,5790350|-58,4335660|
|  3| Palermo 2|Gral. Lucio Norbe...|                CABA|    C deBuenos Aires|-34,5959660|-58,4051500|
|  4|Corrientes| Av. Corrientes 2352|Ciudad de Buenos ...|               Bs As|-34,6046850|-58,3987640|
|  5|   Almagro|      Venezuela 3650|             Capital|             Bs.As. |-34,6173080|-58,4161790|
+---+----------+--------------------+--------------------+--------------------+-----------+-----------+
only showing top 5 rows



In [9]:
cliente.show(5)

+---+------------+--------------------+--------------------+--------+----+-------------------+------------+------------+----------+------------+-------------------------+---------------------------+----------+-----+
| ID|   Provincia|   Nombre_y_Apellido|           Domicilio|Telefono|Edad|          Localidad|           X|           Y|Fecha_Alta|Usuario_Alta|Fecha_Ultima_Modificacion|Usuario_Ultima_Modificacion|Marca_Baja|col10|
+---+------------+--------------------+--------------------+--------+----+-------------------+------------+------------+----------+------------+-------------------------+---------------------------+----------+-----+
|  1|Buenos Aires|  HEBER JONI SANTANA|LAS HERAS Y BAT. ...| 42-5161|  58|         LOMA VERDE|-58,81850307|-34,30997088|2015-01-01|       user1|               2015-01-01|                      user1|         0| NULL|
|  2|Buenos Aires|         ANA SAPRIZA|PUEYRREDON Y DUPU...| 49-7578|  61|         SANTA ROSA|-58,73073751|-34,93908311|2015-01-01|     

***
Visualizando la Inferencia de los Schemas

In [11]:
proveedor.printSchema()

root
 |-- IDProveedor: integer (nullable = true)
 |-- Nombre: string (nullable = true)
 |-- Address: string (nullable = true)
 |-- City: string (nullable = true)
 |-- State: string (nullable = true)
 |-- Country: string (nullable = true)
 |-- departamen: string (nullable = true)



In [12]:
sucursal.printSchema()

root
 |-- ID: integer (nullable = true)
 |-- Sucursal: string (nullable = true)
 |-- Direccion: string (nullable = true)
 |-- Localidad: string (nullable = true)
 |-- Provincia: string (nullable = true)
 |-- Latitud: string (nullable = true)
 |-- Longitud: string (nullable = true)



In [13]:
cliente.printSchema()

root
 |-- ID: integer (nullable = true)
 |-- Provincia: string (nullable = true)
 |-- Nombre_y_Apellido: string (nullable = true)
 |-- Domicilio: string (nullable = true)
 |-- Telefono: string (nullable = true)
 |-- Edad: integer (nullable = true)
 |-- Localidad: string (nullable = true)
 |-- X: string (nullable = true)
 |-- Y: string (nullable = true)
 |-- Fecha_Alta: date (nullable = true)
 |-- Usuario_Alta: string (nullable = true)
 |-- Fecha_Ultima_Modificacion: date (nullable = true)
 |-- Usuario_Ultima_Modificacion: string (nullable = true)
 |-- Marca_Baja: integer (nullable = true)
 |-- col10: string (nullable = true)



***
Llevando Campos a Letra Capital

In [10]:
# TABLA cliente
cliente = cliente.withColumn('Localidad', initcap('Localidad'))
cliente = cliente.withColumn('Provincia', initcap('Provincia'))
cliente = cliente.withColumn('Domicilio', initcap('Domicilio'))
cliente = cliente.withColumn('Nombre_y_Apellido', initcap('Nombre_y_Apellido'))

# TABLA sucursal
sucursal = sucursal.withColumn('Localidad', initcap('Localidad'))
sucursal = sucursal.withColumn('Provincia', initcap('Provincia'))
sucursal = sucursal.withColumn('Direccion', initcap('Direccion'))

# TABLA proveedor
proveedor = proveedor.withColumn('Address', initcap('Address'))
proveedor = proveedor.withColumn('City', initcap('City'))
proveedor = proveedor.withColumn('State', initcap('State'))
proveedor = proveedor.withColumn('Country', initcap('Country'))
proveedor = proveedor.withColumn('departamen', initcap('departamen'))

In [11]:
cliente.show(1)
sucursal.show(1)
proveedor.show(1)

+---+------------+------------------+--------------------+--------+----+----------+------------+------------+----------+------------+-------------------------+---------------------------+----------+-----+
| ID|   Provincia| Nombre_y_Apellido|           Domicilio|Telefono|Edad| Localidad|           X|           Y|Fecha_Alta|Usuario_Alta|Fecha_Ultima_Modificacion|Usuario_Ultima_Modificacion|Marca_Baja|col10|
+---+------------+------------------+--------------------+--------+----+----------+------------+------------+----------+------------+-------------------------+---------------------------+----------+-----+
|  1|Buenos Aires|Heber Joni Santana|Las Heras Y Bat. ...| 42-5161|  58|Loma Verde|-58,81850307|-34,30997088|2015-01-01|       user1|               2015-01-01|                      user1|         0| NULL|
+---+------------+------------------+--------------------+--------+----+----------+------------+------------+----------+------------+-------------------------+---------------------

***
Normalizacion de "Buenos Aires", "Córdoba" y "Capital Federal" y "Río Negro" en los campos  `Localidad` y `Provincia` Tomar en cuenta que en la Tabla proveedor esos campos todavía se llaman `City` y `State`

In [12]:
reemplazar_1 = [
    "Bs.as. ",
    "B.aires",
    "B. Aires",
    "B.Aires",
    " Bs.As.",
    "Bs.As. ",
    "Bs As",
    "Bs.As.",
    "Buenos Aires",
    "C Debuenos Aires",
    "Caba",
    "Cdad De Buenos Aires",
    "Ciudad de Buenos Aires",
    "Ciudad De Buenos Aires",
    "Cdad de Buenos Aires",
    "Pcia Bs AS",
    "Pcia Bs As",
    "Prov de Bs As.",
    "Prov De Bs As.",
    "Prov de Buenos Aires.",
    "Provincia de Buenos Aires",
    "Provincia De Buenos Aires",
    "CABA",
    "C deBuenos Aires"]
# Se reemplazaran por "Buenos Aires"

reemplazar_2 = ["Coroba", "Cordoba",  "Cã³rdoba"]
# Se reemplazaran por "Córdoba"

reemplazar_3 = [
    "Boca De Atencion Monte Castro",
    "Cap.   Federal",
    "Cap. Fed.",
    "Capfed",
    "CapFed",
    "Cap. Fed.",
    "Cap. Federal",
    "Capital"
]
# Se reemplazaran por "Capital Federal"

reemplazar_4 = ["Rio Negro"]
# Se reemplazara por "Río Negro"

Normalizando Buenos Aires

In [13]:
# TABLA `sucursal`
sucursal = sucursal.withColumn('Localidad', when(col('Localidad')\
                               .isin(reemplazar_1), 'Buenos Aires')\
                               .otherwise(col('Localidad')))

sucursal = sucursal.withColumn('Provincia', when(col('Provincia')\
                               .isin(reemplazar_1), 'Buenos Aires')\
                               .otherwise(col('Provincia')))

# TABLA `cliente`
cliente = cliente.withColumn('Localidad', when(col('Localidad')\
                               .isin(reemplazar_1), 'Buenos Aires')\
                               .otherwise(col('Localidad')))

cliente = cliente.withColumn('Provincia', when(col('Provincia')\
                               .isin(reemplazar_1), 'Buenos Aires')\
                               .otherwise(col('Provincia')))

# TABLA `proveedor`
proveedor = proveedor.withColumn('City', when(col('City')\
                               .isin(reemplazar_1), 'Buenos Aires')\
                               .otherwise(col('City')))

proveedor = proveedor.withColumn('State', when(col('State')\
                               .isin(reemplazar_1), 'Buenos Aires')\
                               .otherwise(col('State')))


Normalizando Córdoba

In [14]:

# TABLA `sucursal`
sucursal = sucursal.withColumn('Localidad', when(col('Localidad')\
                               .isin(reemplazar_2), 'Córdoba')\
                               .otherwise(col('Localidad')))

sucursal = sucursal.withColumn('Provincia', when(col('Provincia')\
                               .isin(reemplazar_2), 'Córdoba')\
                               .otherwise(col('Provincia')))

# TABLA `cliente`
cliente = cliente.withColumn('Localidad', when(col('Localidad')\
                               .isin(reemplazar_2), 'Córdoba')\
                               .otherwise(col('Localidad')))

cliente = cliente.withColumn('Provincia', when(col('Provincia')\
                               .isin(reemplazar_2), 'Córdoba')\
                               .otherwise(col('Provincia')))

# TABLA `proveedor`
proveedor = proveedor.withColumn('City', when(col('City')\
                               .isin(reemplazar_2), 'Córdoba')\
                               .otherwise(col('City')))

proveedor = proveedor.withColumn('State', when(col('State')\
                               .isin(reemplazar_2), 'Córdoba')\
                               .otherwise(col('State')))

Normalizando Capital Federal

In [15]:
# TABLA `sucursal`
sucursal = sucursal.withColumn('Localidad', when(col('Localidad')\
                               .isin(reemplazar_3), 'Capital Federal')\
                               .otherwise(col('Localidad')))

sucursal = sucursal.withColumn('Provincia', when(col('Provincia')\
                               .isin(reemplazar_3), 'Capital Federal')\
                               .otherwise(col('Provincia')))

# TABLA `cliente`
cliente = cliente.withColumn('Localidad', when(col('Localidad')\
                               .isin(reemplazar_3), 'Capital Federal')\
                               .otherwise(col('Localidad')))

cliente = cliente.withColumn('Provincia', when(col('Provincia')\
                               .isin(reemplazar_3), 'Capital Federal')\
                               .otherwise(col('Provincia')))

# TABLA `proveedor`
proveedor = proveedor.withColumn('City', when(col('City')\
                               .isin(reemplazar_3), 'Capital Federal')\
                               .otherwise(col('City')))

proveedor = proveedor.withColumn('State', when(col('State')\
                               .isin(reemplazar_3), 'Capital Federal')\
                               .otherwise(col('State')))

Normalizando Río Negro

In [16]:
# TABLA `sucursal`
sucursal = sucursal.withColumn('Localidad', when(col('Localidad')\
                               .isin(reemplazar_4), 'Río Negro')\
                               .otherwise(col('Localidad')))

sucursal = sucursal.withColumn('Provincia', when(col('Provincia')\
                               .isin(reemplazar_4), 'Río Negro')\
                               .otherwise(col('Provincia')))

# TABLA `cliente`
cliente = cliente.withColumn('Localidad', when(col('Localidad')\
                               .isin(reemplazar_4), 'Río Negro')\
                               .otherwise(col('Localidad')))

cliente = cliente.withColumn('Provincia', when(col('Provincia')\
                               .isin(reemplazar_4), 'Río Negro')\
                               .otherwise(col('Provincia')))

# TABLA `proveedor`
proveedor = proveedor.withColumn('City', when(col('City')\
                               .isin(reemplazar_4), 'Río Negro')\
                               .otherwise(col('City')))

proveedor = proveedor.withColumn('State', when(col('State')\
                               .isin(reemplazar_4), 'Río Negro')\
                               .otherwise(col('State')))

In [17]:
del reemplazar_1
del reemplazar_2
del reemplazar_3
del reemplazar_4

In [18]:
cliente.show(1)
sucursal.show(1)
proveedor.show(1)

+---+------------+------------------+--------------------+--------+----+----------+------------+------------+----------+------------+-------------------------+---------------------------+----------+-----+
| ID|   Provincia| Nombre_y_Apellido|           Domicilio|Telefono|Edad| Localidad|           X|           Y|Fecha_Alta|Usuario_Alta|Fecha_Ultima_Modificacion|Usuario_Ultima_Modificacion|Marca_Baja|col10|
+---+------------+------------------+--------------------+--------+----+----------+------------+------------+----------+------------+-------------------------+---------------------------+----------+-----+
|  1|Buenos Aires|Heber Joni Santana|Las Heras Y Bat. ...| 42-5161|  58|Loma Verde|-58,81850307|-34,30997088|2015-01-01|       user1|               2015-01-01|                      user1|         0| NULL|
+---+------------+------------------+--------------------+--------+----+----------+------------+------------+----------+------------+-------------------------+---------------------

***
Normalizando Nombre de Sucursales

In [19]:
sucursal = sucursal.withColumn('Sucursal', when(col('Sucursal')\
                               .isin(["Mendoza1"]), 'Mendoza-1')\
                               .otherwise(col('Sucursal')))

sucursal = sucursal.withColumn('Sucursal', when(col('Sucursal')\
                               .isin(["Mendoza2"]), 'Mendoza-2')\
                               .otherwise(col('Sucursal')))

sucursal = sucursal.withColumn('Sucursal', when(col('Sucursal')\
                               .isin(["Palermo 1"]), 'Palermo-1')\
                               .otherwise(col('Sucursal')))

sucursal = sucursal.withColumn('Sucursal', when(col('Sucursal')\
                               .isin(["Palermo 2"]), 'Palermo-2')\
                               .otherwise(col('Sucursal')))

sucursal = sucursal.withColumn('Sucursal', when(col('Sucursal')\
                               .isin(["MDQ1"]), 'Mdq-1')\
                               .otherwise(col('Sucursal')))

sucursal = sucursal.withColumn('Sucursal', when(col('Sucursal')\
                               .isin(["MDQ2"]), 'Mdq-2')\
                               .otherwise(col('Sucursal')))

sucursal = sucursal.withColumn('Sucursal', when(col('Sucursal')\
                               .isin(["Rosario1"]), 'Rosario-1')\
                               .otherwise(col('Sucursal')))

sucursal = sucursal.withColumn('Sucursal', when(col('Sucursal')\
                               .isin(["Rosario2"]), 'Rosario-2')\
                               .otherwise(col('Sucursal')))

***
Corrigiendo `Latitud` y `Longitud` reemplazando la `,` por `.`

In [20]:
# TABLA sucursal
sucursal = sucursal.withColumn('Latitud', regexp_replace('Latitud', ',', '.'))
sucursal = sucursal.withColumn('Longitud', regexp_replace('Longitud', ',', '.'))
# TABLA cliente
cliente = cliente.withColumn('X', regexp_replace('X', ',', '.'))
cliente = cliente.withColumn('Y', regexp_replace('Y', ',', '.'))

Corrigiendo `Latitud` y `Longitud` casteando columnas al tipo float

In [21]:
# TABLA sucursal
sucursal = sucursal.withColumn('Latitud', sucursal['Latitud'].cast(FloatType()))
sucursal = sucursal.withColumn('Longitud', sucursal['Longitud'].cast(FloatType()))
# TABLA cliente
cliente = cliente.withColumn('X', cliente['X'].cast(FloatType()))
cliente = cliente.withColumn('Y', cliente['Y'].cast(FloatType()))

In [22]:
cliente.show(1)
sucursal.show(1)
proveedor.show(1)

+---+------------+------------------+--------------------+--------+----+----------+----------+---------+----------+------------+-------------------------+---------------------------+----------+-----+
| ID|   Provincia| Nombre_y_Apellido|           Domicilio|Telefono|Edad| Localidad|         X|        Y|Fecha_Alta|Usuario_Alta|Fecha_Ultima_Modificacion|Usuario_Ultima_Modificacion|Marca_Baja|col10|
+---+------------+------------------+--------------------+--------+----+----------+----------+---------+----------+------------+-------------------------+---------------------------+----------+-----+
|  1|Buenos Aires|Heber Joni Santana|Las Heras Y Bat. ...| 42-5161|  58|Loma Verde|-58.818504|-34.30997|2015-01-01|       user1|               2015-01-01|                      user1|         0| NULL|
+---+------------+------------------+--------------------+--------+----+----------+----------+---------+----------+------------+-------------------------+---------------------------+----------+-----+


***
Buscando Valores Faltantes

In [23]:
proveedor.agg(*[count(when(isnull(c), c)).alias(c) for c in proveedor.columns]).show()
cliente.agg(*[count(when(isnull(c), c)).alias(c) for c in cliente.columns]).show()
sucursal.agg(*[count(when(isnull(c), c)).alias(c) for c in sucursal.columns]).show()


+-----------+------+-------+----+-----+-------+----------+
|IDProveedor|Nombre|Address|City|State|Country|departamen|
+-----------+------+-------+----+-----+-------+----------+
|          0|     2|      0|   0|    0|      0|         0|
+-----------+------+-------+----+-----+-------+----------+

+---+---------+-----------------+---------+--------+----+---------+---+---+----------+------------+-------------------------+---------------------------+----------+-----+
| ID|Provincia|Nombre_y_Apellido|Domicilio|Telefono|Edad|Localidad|  X|  Y|Fecha_Alta|Usuario_Alta|Fecha_Ultima_Modificacion|Usuario_Ultima_Modificacion|Marca_Baja|col10|
+---+---------+-----------------+---------+--------+----+---------+---+---+----------+------------+-------------------------+---------------------------+----------+-----+
|  0|       31|               46|       48|      90|   0|       32| 62| 60|         0|           0|                        0|                          0|         0| 3407|
+---+---------+-----

***
Llenando Campos Nulos con "Sin Dato" o 0 segun sea el caso

In [24]:
# TABLA cliente
cliente = cliente.fillna({'Nombre_y_Apellido':'Sin Dato'})
cliente = cliente.fillna({'Domicilio':'Sin Dato'})
cliente = cliente.fillna({'Telefono':'Sin Dato'})
cliente = cliente.fillna({'X':0})
cliente = cliente.fillna({'Y':0})
# TABLA proveedor
proveedor = proveedor.fillna({'Nombre':'Sin Dato'})

***
Filtrar en `cliente` los registros cuya "Localidad" o "Provincia" son Nulas

In [25]:
filtrados = cliente.filter(col('Localidad').isNull() | col('Provincia').isNull())
filtrados.show(32)

+----+---------+--------------------+--------------------+------------+----+---------+----------+----------+----------+------------+-------------------------+---------------------------+----------+-----+
|  ID|Provincia|   Nombre_y_Apellido|           Domicilio|    Telefono|Edad|Localidad|         X|         Y|Fecha_Alta|Usuario_Alta|Fecha_Ultima_Modificacion|Usuario_Ultima_Modificacion|Marca_Baja|col10|
+----+---------+--------------------+--------------------+------------+----+---------+----------+----------+----------+------------+-------------------------+---------------------------+----------+-----+
|  12|     NULL|Eduardo Vicente S...|Carlos Tejedor 44...|   4480-0769|  52|     NULL|-58.579502| -34.82788|2015-01-01|       user1|               2015-01-01|                      user1|         0| NULL|
|  15|     NULL|Jesus Eduardo Sar...|52 Y 235 S/n  Esq...|    496-4503|  25|     NULL|-58.074615|  -35.0316|2015-01-01|       user1|               2015-01-01|                      user

Dado que los registros cuya Localidad y/o Provincia que presentaron valores Nulos, tenían Latitud y Longitud, se buscaron aparte los Nombres de las Localidades y Provincias a las cuales pertenecian

In [26]:
provincias_faltantes = [    
 'Buenos Aires',
 'Buenos Aires',
 'Buenos Aires',
 'Buenos Aires',
 'Buenos Aires',
 'Buenos Aires',
 'Buenos Aires',
 'Buenos Aires',
 'Buenos Aires',
 'Buenos Aires',
 'Buenos Aires',
 'Buenos Aires',
 'Buenos Aires',
 'Buenos Aires',
 'Buenos Aires',
 'Buenos Aires',
 'Buenos Aires',
 'Buenos Aires',
 'Buenos Aires',
 'Buenos Aires',
 'Buenos Aires',
 'Córdoba',
 'Córdoba',
 'Córdoba',
 'Córdoba',
 'Córdoba',
 'Neuquén',
 'Neuquén',
 'Tucumán',
 'Tucumán',
 'Tucumán',
 'Tucumán'
]


localidades_faltantes = [
    'Aeropuerto Internacional Ezeiza',
    'Ángel Etcheverry',
    'Berisso',
    'Guernica',
    'Guernica',
    'La Plata',
    'La Plata',
    'Abasto',
    'La Plata',
    'Adrogué',
    'Adrogué',
    'Adrogué',
    'Pilar Sur',
    'El Cazador',
    'Ruta Nacional 8',
    'Jardín de Infantes 918',
    'Tercera Sección',
    'San Fernando',
    'Ruta Nacional 8',
    'El Peligro',
    'Ruta Nacional 8',
    'Peña',
    'Mariano Mansilla',
    'José Esteban Echeverría',
    'Hugo Juárez',
    'Ruta Nacional 8',
    'San Martín de los Andes',
    'Huilqui Menuco',
    'Alederetes',
    'Tafí Viejo',
    'El Manantial',
    'Santa Bárbara'
]

Creo un Dataframe con los Valores Faltantes y le creo un indice autoincremental

In [27]:
import numpy as np

Se crea un Dataframe con las provincias y localidades faltantes con un indice de 0 a 31

In [28]:
#Index que voy a poner a coincidir con filtrados
indice = np.arange(32).astype(IntegerType) # el indice va de 0 a 31
valores_faltantes_df = spark.createDataFrame(zip(indice, provincias_faltantes, localidades_faltantes),
                                              schema=['index','prov_falta', 'loc_falta'])

In [29]:
del indice

Se le crea a filtrado el indice de 0 a 31 para que coincida con valores_faltantes_df

In [30]:
filtrados = filtrados.withColumn('index', monotonically_increasing_id())

Se unen ambos Dataframes a traves de la columna index y se borra dicha columna

In [31]:
filtrados = filtrados.join(valores_faltantes_df, on='index').drop('index')

In [32]:
filtrados.show(32)

+----+---------+--------------------+--------------------+------------+----+---------+----------+----------+----------+------------+-------------------------+---------------------------+----------+-----+------------+--------------------+
|  ID|Provincia|   Nombre_y_Apellido|           Domicilio|    Telefono|Edad|Localidad|         X|         Y|Fecha_Alta|Usuario_Alta|Fecha_Ultima_Modificacion|Usuario_Ultima_Modificacion|Marca_Baja|col10|  prov_falta|           loc_falta|
+----+---------+--------------------+--------------------+------------+----+---------+----------+----------+----------+------------+-------------------------+---------------------------+----------+-----+------------+--------------------+
|  12|     NULL|Eduardo Vicente S...|Carlos Tejedor 44...|   4480-0769|  52|     NULL|-58.579502| -34.82788|2015-01-01|       user1|               2015-01-01|                      user1|         0| NULL|Buenos Aires|Aeropuerto Intern...|
|  15|     NULL|Jesus Eduardo Sar...|52 Y 235 S/

Ya filtrados tiene los valores faltantes... <br>
Se crea un join con cliente por medio del campo ID

In [33]:
# filtrados = filtrados.drop(filtrados.Localidad).drop(filtrados.Provincia)
filtra2 = filtrados.select("ID",'prov_falta', 'loc_falta')
filtra2 = filtra2.withColumnRenamed("ID", "id-f")

In [34]:
# Realizar un left join entre 'cliente' y 'filtrados'
cliente_join_left = cliente.join(filtra2, cliente['ID'] == filtra2['id-f'], 'left')

# Actualizar las columnas 'Localidad' y 'Provincia' con los valores faltantes
cliente_join_left = cliente_join_left.withColumn('Localidad', coalesce(filtra2['loc_falta'], 
                                                                       cliente['Localidad']))

cliente_join_left = cliente_join_left.withColumn('Provincia', coalesce(filtra2['prov_falta'], 
                                                                       cliente['Provincia']))

# Eliminar las columnas adicionales de 'filtrados' si es necesario
cliente_actualizado = cliente_join_left.drop(filtra2['loc_falta']).drop(filtra2['prov_falta']).drop(filtra2['id-f'])

In [35]:
cliente = cliente_actualizado

In [36]:
del filtra2
del filtrados
del cliente_join_left
del cliente_actualizado
del valores_faltantes_df
del provincias_faltantes
del localidades_faltantes

In [37]:
cliente.show()

+---+------------+--------------------+--------------------+-------------+----+--------------------+----------+----------+----------+------------+-------------------------+---------------------------+----------+-----+
| ID|   Provincia|   Nombre_y_Apellido|           Domicilio|     Telefono|Edad|           Localidad|         X|         Y|Fecha_Alta|Usuario_Alta|Fecha_Ultima_Modificacion|Usuario_Ultima_Modificacion|Marca_Baja|col10|
+---+------------+--------------------+--------------------+-------------+----+--------------------+----------+----------+----------+------------+-------------------------+---------------------------+----------+-----+
| 12|Buenos Aires|Eduardo Vicente S...|Carlos Tejedor 44...|    4480-0769|  52|Aeropuerto Intern...|-58.579502| -34.82788|2015-01-01|       user1|               2015-01-01|                      user1|         0| NULL|
|  1|Buenos Aires|  Heber Joni Santana|Las Heras Y Bat. ...|      42-5161|  58|          Loma Verde|-58.818504| -34.30997|2015-0

***
Se crea un Rango Etario

In [38]:
cliente = cliente.withColumn(
    'RangoEtario',
    when(cliente['Edad'] <= 30, '1_Hasta 30 años')
    .when((cliente['Edad'] > 30) & (cliente['Edad'] <= 40), '2_De 31 a 40 años')
    .when((cliente['Edad'] > 40) & (cliente['Edad'] <= 50), '3_De 41 a 50 años')
    .when((cliente['Edad'] > 50) & (cliente['Edad'] <= 60), '4_De 51 a 60 años')
    .otherwise('5_Desde 61 años')
)

Borro Columna col10

In [39]:
cliente = cliente.drop(cliente.col10)

In [40]:
cliente.show(1)

+---+------------+------------------+--------------------+--------+----+----------+----------+---------+----------+------------+-------------------------+---------------------------+----------+-----------------+
| ID|   Provincia| Nombre_y_Apellido|           Domicilio|Telefono|Edad| Localidad|         X|        Y|Fecha_Alta|Usuario_Alta|Fecha_Ultima_Modificacion|Usuario_Ultima_Modificacion|Marca_Baja|      RangoEtario|
+---+------------+------------------+--------------------+--------+----+----------+----------+---------+----------+------------+-------------------------+---------------------------+----------+-----------------+
|  1|Buenos Aires|Heber Joni Santana|Las Heras Y Bat. ...| 42-5161|  58|Loma Verde|-58.818504|-34.30997|2015-01-01|       user1|               2015-01-01|                      user1|         0|4_De 51 a 60 años|
+---+------------+------------------+--------------------+--------+----+----------+----------+---------+----------+------------+------------------------

***
Empezando a Trabajar la Tabla localidad

In [41]:
# Se crean Todas las combinaciones existentes en las distintas tablas (Pero sin duplicados)
temp_cliente_localidad = cliente.select("Localidad", "Provincia").dropDuplicates()
temp_sucursal_localidad = sucursal.select("Localidad", "Provincia").dropDuplicates()
temp_proveedor_localidad = proveedor.select("City", "State").dropDuplicates()

temp_proveedor_localidad = temp_proveedor_localidad.withColumnRenamed("City", "Localidad")\
                                                   .withColumnRenamed("State", "Provincia")

# Concateno todo en localidad usando union
localidad = temp_cliente_localidad\
            .union(temp_sucursal_localidad)\
            .union(temp_proveedor_localidad)

# Elimino Duplicados
localidad = localidad.dropDuplicates()

In [42]:
del temp_cliente_localidad
del temp_proveedor_localidad
del temp_sucursal_localidad

In [43]:
localidad.show()

+--------------------+------------+
|           Localidad|   Provincia|
+--------------------+------------+
|         Lanus Oeste|Buenos Aires|
|    Villa San Carlos|Buenos Aires|
|         Alta Gracia|     Tucumán|
|           City Bell|Buenos Aires|
|Pedro Molina (gua...|     Mendoza|
|            Ruta Sol|Buenos Aires|
|   Arroyo De La Cruz|Buenos Aires|
|               Gomez|Buenos Aires|
|            Castelar|Buenos Aires|
|               Lujan|Buenos Aires|
|         San Vicente|Buenos Aires|
|Rodeo De La Cruz ...|     Mendoza|
|    Pampa Del Malleo|     Neuquén|
|       Campo De Mayo|Buenos Aires|
|      Villa Libertad|Buenos Aires|
|      Boca Del Tigre|     Tucumán|
|Villa Juan Martin...|Buenos Aires|
|      Parada Orlando|Buenos Aires|
|Ingeniero Juan Allan|Buenos Aires|
|Capilla Del Rosar...|     Mendoza|
+--------------------+------------+
only showing top 20 rows



Se Crea la tabla provincia

In [44]:
provincia = localidad.select("Provincia").dropDuplicates()
provincia = provincia.withColumn('IdProvincia', monotonically_increasing_id()+1)

In [45]:
provincia.show(1)

+---------+-----------+
|Provincia|IdProvincia|
+---------+-----------+
|  Neuquén|          1|
+---------+-----------+
only showing top 1 row



Se termina de crear la tabla localidad

In [46]:
localidad = localidad.withColumn('IdLocalidad', monotonically_increasing_id()+1)
localidad.show(truncate=False)

# Necesito utilizar una tabla con nombre del campo 'Provincia' diferente a la otra tabla
provincia_map = provincia.withColumnRenamed("Provincia", "Provi")

# Realizo el Join de las tablas localidad y provincia (en este caso representada por provincia.map)
localidad_join_provincia = localidad.join(provincia_map, localidad.Provincia == provincia_map.Provi)

# De todo el Join Selecciono las columnas que necesito
localidad = localidad_join_provincia.select("IdLocalidad", "Localidad", "IdProvincia")

# Creo una tabla para Mapear contra `cliente`, `proveedor`, `sucursal`
mapeo = localidad_join_provincia.select("IdLocalidad", "Provincia", "Localidad")
mapeo = mapeo.withColumnRenamed("Provincia", "Provi")\
             .withColumnRenamed("Localidad", "Locali")

+--------------------------------+------------+-----------+
|Localidad                       |Provincia   |IdLocalidad|
+--------------------------------+------------+-----------+
|Lanus Oeste                     |Buenos Aires|1          |
|Villa San Carlos                |Buenos Aires|2          |
|Alta Gracia                     |Tucumán     |3          |
|City Bell                       |Buenos Aires|4          |
|Pedro Molina (guaymallen)       |Mendoza     |5          |
|Ruta Sol                        |Buenos Aires|6          |
|Arroyo De La Cruz               |Buenos Aires|7          |
|Gomez                           |Buenos Aires|8          |
|Castelar                        |Buenos Aires|9          |
|Lujan                           |Buenos Aires|10         |
|San Vicente                     |Buenos Aires|11         |
|Rodeo De La Cruz (guaymallen)   |Mendoza     |12         |
|Pampa Del Malleo                |Neuquén     |13         |
|Campo De Mayo                   |Buenos

In [47]:
del localidad_join_provincia

In [48]:
localidad.show(1)

+-----------+------------------+-----------+
|IdLocalidad|         Localidad|IdProvincia|
+-----------+------------------+-----------+
|        530|Chiquilihuin (rai)|          1|
+-----------+------------------+-----------+
only showing top 1 row



Mapeo a la tabla `sucursal` con "mapeo" para colocarle un IdLocalidad que represente su <br>
combinacion `Localidad - Provincia`

In [49]:
sucursal_join_mapeo = sucursal.join(
    mapeo,
    (mapeo['Provi'] == sucursal['Provincia']) & (mapeo['Locali'] == sucursal['Localidad'])
)
sucursal_join_mapeo.show(1)

+---+--------+-------------+---------+------------+----------+---------+-----------+------------+--------+
| ID|Sucursal|    Direccion|Localidad|   Provincia|   Latitud| Longitud|IdLocalidad|       Provi|  Locali|
+---+--------+-------------+---------+------------+----------+---------+-----------+------------+--------+
| 15|Castelar|Sta Rosa 1845| Castelar|Buenos Aires|-34.639305|-58.65567|          9|Buenos Aires|Castelar|
+---+--------+-------------+---------+------------+----------+---------+-----------+------------+--------+
only showing top 1 row



Selecciono los campos que necesito del Join <br>
Renombro la columna 'ID' a 'IdSucursal'

In [50]:
# Seleccionando los campos que necesito
sucursal = sucursal_join_mapeo.select("ID", "Sucursal", "Direccion", "IdLocalidad", "Latitud", "Longitud")

# Renombrando el Campo "ID"
sucursal = sucursal.withColumnRenamed("ID", "IdSucursal")

In [51]:
sucursal = sucursal.withColumnRenamed("Direccion", "Domicilio")

In [52]:
del sucursal_join_mapeo

In [53]:
sucursal.show()

+----------+------------------+--------------------+-----------+----------+----------+
|IdSucursal|          Sucursal|           Domicilio|IdLocalidad|   Latitud|  Longitud|
+----------+------------------+--------------------+-----------+----------+----------+
|        15|          Castelar|       Sta Rosa 1845|          9|-34.639305| -58.65567|
|        10|             Velez|Av. Juan Bautista...|         45|-34.631256|-58.498062|
|         4|        Corrientes| Av. Corrientes 2352|         45|-34.604687|-58.398766|
|         3|         Palermo-2|Gral. Lucio Norbe...|         45|-34.595966| -58.40515|
|         2|         Palermo-1|      Guatemala 5701|         45|-34.579037|-58.433567|
|         1|           Cabildo|    Av. Cabildo 1342|         45|-34.567806|-58.449574|
|        13|           Caseros|Justo José De Urq...|         87| -34.60595| -58.56332|
|        16|         San Justo|Av. Dr. Ignacio A...|        123| -34.68139| -58.55692|
|        27|Cerro de las Rosas|Av. Rafael N

Mapeo a la tabla `cliente` con "mapeo" para colocarle un IdLocalidad que represente su <br>
combinacion `Localidad - Provincia`

In [54]:
cliente_join_mapeo = cliente.join(
    mapeo,
    (mapeo['Provi'] == cliente['Provincia']) & (mapeo['Locali'] == cliente['Localidad'])
)
cliente_join_mapeo.show(1)

+---+------------+--------------------+--------------------+---------+----+-----------+----------+----------+----------+------------+-------------------------+---------------------------+----------+-----------------+-----------+------------+-----------+
| ID|   Provincia|   Nombre_y_Apellido|           Domicilio| Telefono|Edad|  Localidad|         X|         Y|Fecha_Alta|Usuario_Alta|Fecha_Ultima_Modificacion|Usuario_Ultima_Modificacion|Marca_Baja|      RangoEtario|IdLocalidad|       Provi|     Locali|
+---+------------+--------------------+--------------------+---------+----+-----------+----------+----------+----------+------------+-------------------------+---------------------------+----------+-----------------+-----------+------------+-----------+
|393|Buenos Aires|De Chiaria, Claud...|Ministro Brin Y A...|4249-4641|  31|Lanus Oeste|-58.396244|-34.700245|2015-01-01|       user1|               2015-01-01|                      user1|         0|2_De 31 a 40 años|          1|Buenos Air

In [55]:
#Borro los campos que sobran
cliente_join_mapeo = cliente_join_mapeo.drop("Provincia","Localidad","Provi","Locali")

#Renombro Campos
cliente_join_mapeo = cliente_join_mapeo.withColumnRenamed("ID", "IdCliente")\
                                       .withColumnRenamed("X", "Longitud")\
                                       .withColumnRenamed("Y", "Latitud")\
                                       .withColumnRenamed("Nombre_y_Apellido", "NombreApellido")

In [56]:
cliente_join_mapeo.show()

+---------+--------------------+--------------------+---------+----+----------+----------+----------+------------+-------------------------+---------------------------+----------+-----------------+-----------+
|IdCliente|      NombreApellido|           Domicilio| Telefono|Edad|  Longitud|   Latitud|Fecha_Alta|Usuario_Alta|Fecha_Ultima_Modificacion|Usuario_Ultima_Modificacion|Marca_Baja|      RangoEtario|IdLocalidad|
+---------+--------------------+--------------------+---------+----+----------+----------+----------+------------+-------------------------+---------------------------+----------+-----------------+-----------+
|      393|De Chiaria, Claud...|Ministro Brin Y A...|4249-4641|  31|-58.396244|-34.700245|2015-01-01|       user1|               2015-01-01|                      user1|         0|2_De 31 a 40 años|          1|
|      365|Elisa Margarita Z...|1âº De Mayo E/pil...|4262-1887|  53| 58.428944| 34.698895|2015-01-01|       user1|               2015-01-01|                    

In [57]:
cliente = cliente_join_mapeo

In [59]:
cliente = (cliente.withColumnRenamed("Usuario_Alta", "UsuarioAlta")\
                 .withColumnRenamed("Fecha_Alta", "FechaAlta")\
                 .withColumnRenamed("Fecha_Ultima_Modificacion", "FechaUpdate")\
                 .withColumnRenamed("Usuario_Ultima_Modificacion", "UsuarioUpdate")\
                 .withColumnRenamed("Marca_baja", "MarcaBaja"))


In [60]:
del cliente_join_mapeo

In [61]:
cliente.show(5)

+---------+--------------------+--------------------+---------+----+----------+----------+----------+-----------+-----------+-------------+---------+-----------------+-----------+
|IdCliente|      NombreApellido|           Domicilio| Telefono|Edad|  Longitud|   Latitud| FechaAlta|UsuarioAlta|FechaUpdate|UsuarioUpdate|MarcaBaja|      RangoEtario|IdLocalidad|
+---------+--------------------+--------------------+---------+----+----------+----------+----------+-----------+-----------+-------------+---------+-----------------+-----------+
|      393|De Chiaria, Claud...|Ministro Brin Y A...|4249-4641|  31|-58.396244|-34.700245|2015-01-01|      user1| 2015-01-01|        user1|        0|2_De 31 a 40 años|          1|
|      365|Elisa Margarita Z...|1âº De Mayo E/pil...|4262-1887|  53| 58.428944| 34.698895|2015-01-01|      user1| 2015-01-01|        user1|        0|4_De 51 a 60 años|          1|
|      342|Boris Silvestre Y...|Aristobulo Del Va...|4241-2143|  17|  -58.4129|-34.703426|2015-01-01

Mapeo a la tabla `proveedor` con "mapeo" para colocarle un IdLocalidad que represente su <br>
combinacion `Localidad - Provincia`

In [62]:
proveedor_join_mapeo = proveedor.join(
    mapeo,
    (mapeo['Provi'] == proveedor['State']) & (mapeo['Locali'] == proveedor['City'])
)
proveedor_join_mapeo.show(1)

+-----------+--------------+---------------+---------------+------------+---------+----------+-----------+------------+---------------+
|IDProveedor|        Nombre|        Address|           City|       State|  Country|departamen|IdLocalidad|       Provi|         Locali|
+-----------+--------------+---------------+---------------+------------+---------+----------+-----------+------------+---------------+
|          4|Rivero Insumos|Pte. Peron 3872|Valentin Alsina|Buenos Aires|Argentina|     Lanus|         97|Buenos Aires|Valentin Alsina|
+-----------+--------------+---------------+---------------+------------+---------+----------+-----------+------------+---------------+
only showing top 1 row



In [63]:
# Borro los campos que sobran
proveedor_join_mapeo = proveedor_join_mapeo.drop("City", "State", "Provi", "Locali")

# Renombro Campos
proveedor_join_mapeo = (proveedor_join_mapeo.withColumnRenamed("IDProveedor", "IdProveedor")
                                             .withColumnRenamed("Address", "Domicilio")
                                             .withColumnRenamed("Country", "Pais")
                                             .withColumnRenamed("departamen", "Departamento"))

proveedor_join_mapeo.show(1)

+-----------+--------------+---------------+---------+------------+-----------+
|IdProveedor|        Nombre|      Domicilio|     Pais|Departamento|IdLocalidad|
+-----------+--------------+---------------+---------+------------+-----------+
|          4|Rivero Insumos|Pte. Peron 3872|Argentina|       Lanus|         97|
+-----------+--------------+---------------+---------+------------+-----------+
only showing top 1 row



In [64]:
proveedor = proveedor_join_mapeo

In [65]:
del proveedor_join_mapeo

In [66]:
del provincia_map
del mapeo

In [67]:
proveedor.show(1)

+-----------+--------------+---------------+---------+------------+-----------+
|IdProveedor|        Nombre|      Domicilio|     Pais|Departamento|IdLocalidad|
+-----------+--------------+---------------+---------+------------+-----------+
|          4|Rivero Insumos|Pte. Peron 3872|Argentina|       Lanus|         97|
+-----------+--------------+---------------+---------+------------+-----------+
only showing top 1 row



*** 
Toca Imputar valores a las Longitudes y Latitudes NULAS a la cuales ya se les asigno 0 para identificarlas <br>
El acuerdo en clase fue Segun su localidad, ubicar el resto de clientes que tuvieran esa misma localidad y obtener el promedio de las latitudes y longitudes

Necesito Sacar un Nuevo Df a partir de Cliente Solo con los campos 'IdCliente' y 'Latitud' siempre y cuando Latitud sea diferente de cero(0) 

In [68]:
latitudes = cliente.filter(cliente['Latitud'] != 0).select('IdLocalidad', 'Latitud')

In [69]:
# Saco el Promedio de las latitudes por cada IdLocalidad
promedio_latitudes = latitudes.groupBy('IdLocalidad').agg(avg('Latitud').alias('LatitudPromedio'))
promedio_latitudes.show(1)

+-----------+------------------+
|IdLocalidad|   LatitudPromedio|
+-----------+------------------+
|         26|-34.93217468261719|
+-----------+------------------+
only showing top 1 row



In [70]:
# Hago un Join left para tener en una columna la 'latitudPromedio'
cliente = cliente.join(promedio_latitudes, 'IdLocalidad', 'left')

In [71]:
# Mapeo las 'Laltitud` == 0 y hago que sean igaul a `LatitudPromedio`
cliente = cliente.withColumn('Latitud', when(col('Latitud') == 0, 
                             col('LatitudPromedio')).otherwise(col('Latitud')))
cliente.show(1)

+-----------+---------+--------------------+--------------------+---------+----+----------+------------------+----------+-----------+-----------+-------------+---------+-----------------+-------------------+
|IdLocalidad|IdCliente|      NombreApellido|           Domicilio| Telefono|Edad|  Longitud|           Latitud| FechaAlta|UsuarioAlta|FechaUpdate|UsuarioUpdate|MarcaBaja|      RangoEtario|    LatitudPromedio|
+-----------+---------+--------------------+--------------------+---------+----+----------+------------------+----------+-----------+-----------+-------------+---------+-----------------+-------------------+
|          1|      393|De Chiaria, Claud...|Ministro Brin Y A...|4249-4641|  31|-58.396244|-34.70024490356445|2015-01-01|      user1| 2015-01-01|        user1|        0|2_De 31 a 40 años|-29.144632568359373|
+-----------+---------+--------------------+--------------------+---------+----+----------+------------------+----------+-----------+-----------+-------------+---------

In [72]:
# Elimino la Columna 'LatitudPromedio``
cliente = cliente.drop('LatitudPromedio')

In [73]:
del latitudes
del promedio_latitudes

Repito el Proceso pero ahora con 'Longitud'

In [74]:
longitudes = cliente.filter(cliente['Longitud'] != 0).select('IdLocalidad', 'Longitud')

In [75]:
promedio_longitudes = longitudes.groupBy('IdLocalidad').agg(avg('longitud').alias('longitudPromedio'))
promedio_longitudes.show(1)

+-----------+------------------+
|IdLocalidad|  longitudPromedio|
+-----------+------------------+
|         26|-58.00044250488281|
+-----------+------------------+
only showing top 1 row



In [76]:
# Hago un Join left para tener en una columna la 'longitudPromedio'
cliente = cliente.join(promedio_longitudes, 'IdLocalidad', 'left')

In [77]:
# Mapeo las 'Longitud` == 0 y hago que sean igaul a `LongitudPromedio`
cliente = cliente.withColumn('Longitud', when(col('Longitud') == 0, 
                             col('LongitudPromedio')).otherwise(col('Longitud')))
cliente.show(1)

+-----------+---------+--------------------+--------------------+---------+----+-------------------+------------------+----------+-----------+-----------+-------------+---------+-----------------+------------------+
|IdLocalidad|IdCliente|      NombreApellido|           Domicilio| Telefono|Edad|           Longitud|           Latitud| FechaAlta|UsuarioAlta|FechaUpdate|UsuarioUpdate|MarcaBaja|      RangoEtario|  longitudPromedio|
+-----------+---------+--------------------+--------------------+---------+----+-------------------+------------------+----------+-----------+-----------+-------------+---------+-----------------+------------------+
|          1|      393|De Chiaria, Claud...|Ministro Brin Y A...|4249-4641|  31|-58.396244049072266|-34.70024490356445|2015-01-01|      user1| 2015-01-01|        user1|        0|2_De 31 a 40 años|-49.06735595703125|
+-----------+---------+--------------------+--------------------+---------+----+-------------------+------------------+----------+------

In [78]:
# Elimino la Columna 'LongitudPromedio``
cliente = cliente.drop('LongitudPromedio')
cliente.show(1)

+-----------+---------+--------------------+--------------------+---------+----+-------------------+------------------+----------+-----------+-----------+-------------+---------+-----------------+
|IdLocalidad|IdCliente|      NombreApellido|           Domicilio| Telefono|Edad|           Longitud|           Latitud| FechaAlta|UsuarioAlta|FechaUpdate|UsuarioUpdate|MarcaBaja|      RangoEtario|
+-----------+---------+--------------------+--------------------+---------+----+-------------------+------------------+----------+-----------+-----------+-------------+---------+-----------------+
|          1|      393|De Chiaria, Claud...|Ministro Brin Y A...|4249-4641|  31|-58.396244049072266|-34.70024490356445|2015-01-01|      user1| 2015-01-01|        user1|        0|2_De 31 a 40 años|
+-----------+---------+--------------------+--------------------+---------+----+-------------------+------------------+----------+-----------+-----------+-------------+---------+-----------------+
only showing to

In [79]:
cliente.filter(cliente['Longitud'] == 0).show()

+-----------+---------+--------------+---------+--------+----+--------+-------+---------+-----------+-----------+-------------+---------+-----------+
|IdLocalidad|IdCliente|NombreApellido|Domicilio|Telefono|Edad|Longitud|Latitud|FechaAlta|UsuarioAlta|FechaUpdate|UsuarioUpdate|MarcaBaja|RangoEtario|
+-----------+---------+--------------+---------+--------+----+--------+-------+---------+-----------+-----------+-------------+---------+-----------+
+-----------+---------+--------------+---------+--------+----+--------+-------+---------+-----------+-----------+-------------+---------+-----------+



In [80]:
cliente.filter(cliente['Latitud'] == 0).show()

+-----------+---------+--------------+---------+--------+----+--------+-------+---------+-----------+-----------+-------------+---------+-----------+
|IdLocalidad|IdCliente|NombreApellido|Domicilio|Telefono|Edad|Longitud|Latitud|FechaAlta|UsuarioAlta|FechaUpdate|UsuarioUpdate|MarcaBaja|RangoEtario|
+-----------+---------+--------------+---------+--------+----+--------+-------+---------+-----------+-----------+-------------+---------+-----------+
+-----------+---------+--------------+---------+--------+----+--------+-------+---------+-----------+-----------+-------------+---------+-----------+



Tomar en cuenta que hay 3 clientes Los cuales son Unicos en sus Localidades por lo que es imposible calcular Promedio de otros clientes

In [81]:
cliente.agg(*[count(when(isnull(c), c)).alias(c) for c in cliente.columns]).show()

+-----------+---------+--------------+---------+--------+----+--------+-------+---------+-----------+-----------+-------------+---------+-----------+
|IdLocalidad|IdCliente|NombreApellido|Domicilio|Telefono|Edad|Longitud|Latitud|FechaAlta|UsuarioAlta|FechaUpdate|UsuarioUpdate|MarcaBaja|RangoEtario|
+-----------+---------+--------------+---------+--------+----+--------+-------+---------+-----------+-----------+-------------+---------+-----------+
|          0|        0|             0|        0|       0|   0|       3|      3|        0|          0|          0|            0|        0|          0|
+-----------+---------+--------------+---------+--------+----+--------+-------+---------+-----------+-----------+-------------+---------+-----------+



In [82]:
latitudes_nulas = cliente.filter(col('Latitud').isNull()).select('IdLocalidad')
latitudes_nulas.show()

+-----------+
|IdLocalidad|
+-----------+
|        420|
|        511|
|        263|
+-----------+



In [83]:
longitudes_nulas = cliente.filter(col('longitud').isNull()).select('IdLocalidad')
longitudes_nulas.show()

+-----------+
|IdLocalidad|
+-----------+
|        420|
|        511|
|        263|
+-----------+



In [84]:
localidad.filter(col("IdLocalidad") == 263).show()

+-----------+-----------+-----------+
|IdLocalidad|  Localidad|IdProvincia|
+-----------+-----------+-----------+
|        263|Villa Brown|          6|
+-----------+-----------+-----------+



In [85]:
localidad.filter(col("IdLocalidad") == 420).show()

+-----------+----------------+-----------+
|IdLocalidad|       Localidad|IdProvincia|
+-----------+----------------+-----------+
|        420|Angel Etcheverry|          6|
+-----------+----------------+-----------+



In [86]:
localidad.filter(col("IdLocalidad") == 511).show()

+-----------+------------------+-----------+
|IdLocalidad|         Localidad|IdProvincia|
+-----------+------------------+-----------+
|        511|Parque San Lorenzo|          6|
+-----------+------------------+-----------+



In [87]:
provincia.filter(col("IdProvincia") == 6).show()

+------------+-----------+
|   Provincia|IdProvincia|
+------------+-----------+
|Buenos Aires|          6|
+------------+-----------+



In [88]:
del longitudes_nulas
del latitudes_nulas

Se busco por internet manualmente la Longitud y Latitud Promedio de esas 3 Localidades

+ cliente con "IdLocalidad" == 420
    + Angel Etcheverry
    + "Latitud" = -34.56403
    + "Longitud" = -58.53444

+ cliente "IdLocalidad" == 263
    + Villa Brown
    + "Latitud"  = -35.0272
    + "Longitud" = -58.0719

+ cliente con "IdLocalidad" == 511
    + Parque San Lorenzo
    + "Latitud" = -34.87164
    + "Longitud" = -58.28075

In [89]:
# Cliente con IdLocalidad = 420
cliente = cliente.withColumn('Latitud', when(col('IdLocalidad') == 420, -34.56403).otherwise(col('Latitud')))
cliente = cliente.withColumn('Longitud', when(col('IdLocalidad') == 420, -58.53444).otherwise(col('Longitud')))

# Cliente con IdLocalidad = 263
cliente = cliente.withColumn('Latitud', when(col('IdLocalidad') == 263, -35.0272).otherwise(col('Latitud')))
cliente = cliente.withColumn('Longitud', when(col('IdLocalidad') == 263, -58.0719).otherwise(col('Longitud')))

# Cliente con IdLocalidad = 511
cliente = cliente.withColumn('Latitud', when(col('IdLocalidad') == 511, -34.87164).otherwise(col('Latitud')))
cliente = cliente.withColumn('Longitud', when(col('IdLocalidad') == 511, -58.28075).otherwise(col('Longitud')))

In [90]:
cliente.show(1)

+-----------+---------+--------------------+--------------------+---------+----+-------------------+------------------+----------+-----------+-----------+-------------+---------+-----------------+
|IdLocalidad|IdCliente|      NombreApellido|           Domicilio| Telefono|Edad|           Longitud|           Latitud| FechaAlta|UsuarioAlta|FechaUpdate|UsuarioUpdate|MarcaBaja|      RangoEtario|
+-----------+---------+--------------------+--------------------+---------+----+-------------------+------------------+----------+-----------+-----------+-------------+---------+-----------------+
|          1|      393|De Chiaria, Claud...|Ministro Brin Y A...|4249-4641|  31|-58.396244049072266|-34.70024490356445|2015-01-01|      user1| 2015-01-01|        user1|        0|2_De 31 a 40 años|
+-----------+---------+--------------------+--------------------+---------+----+-------------------+------------------+----------+-----------+-----------+-------------+---------+-----------------+
only showing to

***
Creando la Conexion

In [72]:
engine = connec_db()

***
Pasando los Dataframes a Dataframes de Pandas de a uno para evitar colapso

In [91]:
# IMPORTANDO LIBRERIA PANDAS
import pandas as pd

### Pasando Los Dataframes a Pandas para poder pasarlos a Tablas SQL

***
Vaciando el Dataframe a la Tabla `provincia`

In [92]:
pandas_df_provincia = provincia.toPandas()

In [75]:
pandas_df_provincia.to_sql(name="provincia", con=engine, index=False, if_exists="append")

9

In [93]:
pandas_df_provincia.to_csv("csv_practica/provincia.csv", index=False)

In [94]:
del pandas_df_provincia
del provincia

***
Vaciando el Dataframe a la Tabla `localidad`

In [95]:
pandas_df_localidad = localidad.toPandas()

In [78]:
pandas_df_localidad.to_sql(name="localidad", con=engine, index=False, if_exists="append")

607

In [96]:
pandas_df_localidad.to_csv("csv_practica/localidad.csv", index=False)

In [97]:
del localidad
del pandas_df_localidad

***
Vaciando el Dataframe a la Tabla `sucursal`

In [98]:
pandas_df_sucursal = sucursal.toPandas()

In [81]:
pandas_df_sucursal.to_sql(name="sucursal", con=engine, index=False, if_exists="append")

31

In [99]:
pandas_df_sucursal.to_csv("csv_practica/sucursal.csv", index=False)

In [ ]:
pandas_df_sucursal.to_csv("Data/sucursal_normalizada.csv", index=False)

In [100]:
del pandas_df_sucursal
del sucursal

***
Vaciando el Dataframe a la Tabla `proveedor`

In [101]:
pandas_df_proveedor = proveedor.toPandas()

In [84]:
pandas_df_proveedor.to_sql(name="proveedor", con=engine, index=False, if_exists="append")

14

In [102]:
pandas_df_proveedor.to_csv("csv_practica/proveedor.csv", index=False)

In [103]:
del pandas_df_proveedor
del proveedor

***
Vaciando el Dataframe a la Tabla `cliente`

In [104]:
pandas_df_cliente = cliente.toPandas()

In [87]:
pandas_df_cliente.to_sql(name="cliente", con=engine, index=False, if_exists="append")

3407

In [105]:
pandas_df_cliente.to_csv("csv_practica/cliente.csv", index=False)

In [106]:
del cliente
del pandas_df_cliente

In [107]:
spark.stop()